# 条件付き確率場(Conditional Random Field;CRF)を用いた固有表現認識器の構築
条件付き確率場を用いて固有表現認識器の構築を行う. データは固有表現認識の典型的なデータであるCoNLL-03データセットを用いる. 各データは単語ごとに固有表現か非固有表現かのラベル付けがIOB2表記で行われている. 固有表現の場合の始まりのときB, 固有表現の途中のときI, 非固有表現のときOを表す. このようなラベル付けをされたデータに対してテキスト分類と同様にCRFで学習を行う.

# 準備

In [1]:
!pip install sklearn-crfsuite seqeval eli5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.0/106.0 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.1 MB/s eta 0:00:00a 0:00:01
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=a8672598ae56a42521c3d957f1bd3667880b6d17880e3679b11ef06a1157e424
  Stored in directory: /root/.cache/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
Successfully built seqeval
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import eli5
import scipy
from seqeval.metrics import classification_report, f1_score
from sklearn_crfsuite import CRF
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV

/usr/local/lib/python3.8/dist-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# データセット読み込み
データセットは次に示す構造である. まず開始を表す"-DOCSTART"から始まる行がある. 次にスペースの行からスペースの行までの間に1文に関するデータが格納されている. データは単語, 品詞, 構文チャンクタグ, IOB2表記 という風になっている. ここでは単語, 品詞を特徴量, IOB2表記をターゲットとして用いる.
```
-DOCSTART- -X- -X- O

EU NNP B-NP B-ORG
rejects VBZ B-VP O
German JJ B-NP B-MISC
call NN I-NP O
to TO B-VP O
boycott VB I-VP O
British JJ B-NP B-MISC
lamb NN I-NP O
. . O O

Peter NNP B-NP B-PER
Blackburn NNP I-NP I-PER

BRUSSELS NNP B-NP B-LOC
1996-08-22 CD I-NP O
```


In [5]:
def load_conll(file_path):
    sents = []
    sent = []
    with open(file_path,encoding="utf-8") as f:
        for line in f:
            line = line.strip() # 文字間の空白以外の空白文字を削除
            if line.startswith("-DOCSTART"):
                continue
            if line:
                word,pos,_,tag=line.split()
                sent.append((word,pos,tag))
            else:
                if len(sent)==0:
                    continue
                sents.append(sent)
                sent = []
    return sents

In [6]:
train_sents = load_conll("./data/conll2003/en/train.txt")
valid_sents = load_conll("./data/conll2003/en/valid.txt")
test_sents = load_conll("./data/conll2003/en/test.txt")

In [11]:
train_sents[:3]

[[('EU', 'NNP', 'B-ORG'),
  ('rejects', 'VBZ', 'O'),
  ('German', 'JJ', 'B-MISC'),
  ('call', 'NN', 'O'),
  ('to', 'TO', 'O'),
  ('boycott', 'VB', 'O'),
  ('British', 'JJ', 'B-MISC'),
  ('lamb', 'NN', 'O'),
  ('.', '.', 'O')],
 [('Peter', 'NNP', 'B-PER'), ('Blackburn', 'NNP', 'I-PER')],
 [('BRUSSELS', 'NNP', 'B-LOC'), ('1996-08-22', 'CD', 'O')]]

In [13]:
print(len(train_sents))
print(len(valid_sents))
print(len(test_sents))

14041
3250
3453


# 前処理
前処理として次の5つを注目単語の前後2単語(ウィンドウサイズ2のコンテキスト)に関して調べる処理を行う.
- 小文字化した単語
- 大文字だけからなる単語か
- 単語の先頭の文字は大文字か
- 数字か
- 品詞

特徴量の生成に用いる関数の凡例
```python
test_str1 = "Apple"
test_str2 = "GREEN"
test_str3 = "1234"
print(test_str1.lower())
print(test_str1.isupper())
print(test_str2.isupper())
print(test_str1.istitle())
print(test_str2.istitle())
print(test_str1.isdigit())
print(test_str3.isdigit())
```

実行結果
```
apple
False
True
True
False
False
True
```

In [23]:
def word2features(sent,i):
    word=sent[i][0]
    postag = sent[i][1]
    
    # 注目単語の特徴量
    features = {
    "bias":1.0,
    "word.lower()":word.lower(),
    "word.isupper":word.isupper(),
    "word.istitle":word.istitle(),
    "word.isdigit":word.isdigit(),
    "postag":postag,
    }
    
    # 注目単語の1つ前の単語
    if i>0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
        "-1:word.lower()":word1.lower(),
        "-1:word.isupper":word1.isupper(),
        "-1:word.istitle":word1.istitle(),
        "-1:postag":postag1,
        })
    else: # BOSのとき
        features["BOS"] = True
    
    # 注目単語の2つ前の単語
    if i>1:
        word2 = sent[i-2][0]
        postag2 = sent[i-2][1]
        features.update({
        "-2:word.lower()":word2.lower(),
        "-2:word.isupper":word2.isupper(),
        "-2:word.istitle":word2.istitle(),
        "-2:postag":postag2,
        })
    else: # BOSのとき
        features["-2:BOS"] = True
        
    # 注目単語の1つ後の単語
    if i<len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
        "+1:word.lower()":word1.lower(),
        "+1:word.isupper":word1.isupper(),
        "+1:word.istitle":word1.istitle(),
        "+1:postag":postag1,
        })
    else: # EOSのとき
        features["EOS"] = True
    
    # 注目単語の2つ後の単語
    if i<len(sent)-2:
        word2 = sent[i+2][0]
        postag2 = sent[i+2][1]
        features.update({
        "+2:word.lower()":word2.lower(),
        "+2:word.isupper":word2.isupper(),
        "+2:word.istitle":word2.istitle(),
        "+2:postag":postag2,
        })
    else: # EOSのとき
        features["+2:EOS"] = True
        
    return features

def sent2features(sent):
    """引数で与えられたsentのすべてに対して特徴量を計算"""
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [24]:
sent2features(train_sents[0])[0] # "EU"の特徴量を計算

{'bias': 1.0,
 'word.lower()': 'eu',
 'word.isupper': True,
 'word.istitle': False,
 'word.isdigit': False,
 'postag': 'NNP',
 'BOS': True,
 '-2:BOS': True,
 '+1:word.lower()': 'rejects',
 '+1:word.isupper': False,
 '+1:word.istitle': False,
 '+1:postag': 'VBZ',
 '+2:word.lower()': 'german',
 '+2:word.isupper': False,
 '+2:word.istitle': True,
 '+2:postag': 'JJ'}

In [25]:
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_valid = [sent2features(s) for s in valid_sents]
y_valid = [sent2labels(s) for s in valid_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

In [27]:
X_train[:1]

[[{'bias': 1.0,
   'word.lower()': 'eu',
   'word.isupper': True,
   'word.istitle': False,
   'word.isdigit': False,
   'postag': 'NNP',
   'BOS': True,
   '-2:BOS': True,
   '+1:word.lower()': 'rejects',
   '+1:word.isupper': False,
   '+1:word.istitle': False,
   '+1:postag': 'VBZ',
   '+2:word.lower()': 'german',
   '+2:word.isupper': False,
   '+2:word.istitle': True,
   '+2:postag': 'JJ'},
  {'bias': 1.0,
   'word.lower()': 'rejects',
   'word.isupper': False,
   'word.istitle': False,
   'word.isdigit': False,
   'postag': 'VBZ',
   '-1:word.lower()': 'eu',
   '-1:word.isupper': True,
   '-1:word.istitle': False,
   '-1:postag': 'NNP',
   '-2:BOS': True,
   '+1:word.lower()': 'german',
   '+1:word.isupper': False,
   '+1:word.istitle': True,
   '+1:postag': 'JJ',
   '+2:word.lower()': 'call',
   '+2:word.isupper': False,
   '+2:word.istitle': False,
   '+2:postag': 'NN'},
  {'bias': 1.0,
   'word.lower()': 'german',
   'word.isupper': False,
   'word.istitle': True,
   'word.isd

# Modeling

In [29]:
model = CRF(
algorithm="lbfgs",
max_iterations=100,
all_possible_transitions=False)

# validもtrainにして学習
try:
    model.fit(X_train+X_valid,y_train+y_valid)
except AttributeError:
    pass

In [30]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

         LOC     0.8385    0.7842    0.8104      1668
        MISC     0.7602    0.6638    0.7087       702
         ORG     0.7256    0.7038    0.7145      1661
         PER     0.8041    0.8683    0.8350      1617

   micro avg     0.7861    0.7697    0.7778      5648
   macro avg     0.7821    0.7550    0.7672      5648
weighted avg     0.7857    0.7697    0.7766      5648



In [47]:
# 予測例を表示
text_idx = 0
for idx in range(len(test_sents[text_idx])):
    print(test_sents[text_idx][idx],"pred :",y_pred[text_idx][idx],", ans :",y_test[text_idx][idx])

('SOCCER', 'NN', 'O') pred : O , ans : O
('-', ':', 'O') pred : O , ans : O
('JAPAN', 'NNP', 'B-LOC') pred : B-LOC , ans : B-LOC
('GET', 'VB', 'O') pred : O , ans : O
('LUCKY', 'NNP', 'O') pred : B-ORG , ans : O
('WIN', 'NNP', 'O') pred : O , ans : O
(',', ',', 'O') pred : O , ans : O
('CHINA', 'NNP', 'B-PER') pred : B-LOC , ans : B-PER
('IN', 'IN', 'O') pred : O , ans : O
('SURPRISE', 'DT', 'O') pred : O , ans : O
('DEFEAT', 'NN', 'O') pred : O , ans : O
('.', '.', 'O') pred : O , ans : O
